In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from tensorflow.keras.models import Model, Sequential
from keras.applications.nasnet import NASNetLarge
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,
                                  
        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255
                                  )

In [3]:
train_dataset  = train_datagen.flow_from_directory(directory = 'C:/Users/unknown/Downloads/Emotion_detection/Emotion_detection/Emotion_data/train',
                                                   target_size = (48,48),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   batch_size = 64)

Found 23060 images belonging to 7 classes.


In [4]:
valid_dataset = valid_datagen.flow_from_directory(directory = 'C:/Users/unknown/Downloads/Emotion_detection/Emotion_detection/Emotion_data/train',
                                                  target_size = (48,48),
                                                  class_mode = 'categorical',
                                                  subset = 'validation',
                                                  batch_size = 64)

Found 5761 images belonging to 7 classes.


In [5]:
test_dataset = test_datagen.flow_from_directory(directory = 'C:/Users/unknown/Downloads/Emotion_detection/Emotion_detection/Emotion_data/validation',
                                                  target_size = (48,48),
                                                  class_mode = 'categorical',
                                                  batch_size = 64)

Found 7066 images belonging to 7 classes.


In [10]:
base_model = tf.keras.applications.VGG16(input_shape=(48,48,3),include_top=False,weights="imagenet")

In [11]:
# Freezing Layers

for layer in base_model.layers[:-2]:
    layer.trainable=False

In [16]:
# Building Model

model2=Sequential()
model2.add(base_model)
model2.add(Dropout(0.5))
model2.add(Flatten())
model2.add(BatchNormalization())
model2.add(Dense(32,kernel_initializer='he_uniform'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(32,kernel_initializer='he_uniform'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(32,kernel_initializer='he_uniform'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dense(7,activation='softmax'))

In [17]:
# Model Summary

model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 dropout_6 (Dropout)         (None, 1, 1, 512)         0         
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 batch_normalization_8 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 32)                16416     
                                                                 
 batch_normalization_9 (Batc  (None, 32)               128       
 hNormalization)                                      

In [18]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [19]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]

In [20]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 20,verbose = 1,factor = 0.50, min_lr = 1e-10)

mcp = ModelCheckpoint('model2.h5')

es = EarlyStopping(verbose=1, patience=20)

In [21]:
model2.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=METRICS)

In [22]:
history=model2.fit(train_dataset,validation_data=valid_dataset,epochs = 60,verbose = 1,callbacks=[lrd,mcp,es])

Epoch 1/60
361/361 [==============================] - 296s 785ms/step - loss: 1.8875 - accuracy: 0.8556 - precision: 0.1949 - recall: 0.0033 - auc: 0.6217 - f1_score: 0.0064 - val_loss: 1.7651 - val_accuracy: 0.8571 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6995 - val_f1_score: 0.0000e+00 - lr: 0.0010
Epoch 2/60
361/361 [==============================] - 281s 777ms/step - loss: 1.7990 - accuracy: 0.8570 - precision: 0.4565 - recall: 0.0055 - auc: 0.6637 - f1_score: 0.0107 - val_loss: 1.7267 - val_accuracy: 0.8572 - val_precision: 1.0000 - val_recall: 5.2074e-04 - val_auc: 0.7114 - val_f1_score: 0.0010 - lr: 0.0010
Epoch 3/60
361/361 [==============================] - 286s 791ms/step - loss: 1.7692 - accuracy: 0.8572 - precision: 0.5155 - recall: 0.0094 - auc: 0.6800 - f1_score: 0.0182 - val_loss: 1.7063 - val_accuracy: 0.8575 - val_precision: 1.0000 - val_recall: 0.0023 - val_auc: 0.7171 - val_f1_score: 0.0044 - lr: 0.0010
Epoch 4/60
361/361 [===================

Epoch 27/60
361/361 [==============================] - 322s 892ms/step - loss: 1.6886 - accuracy: 0.8588 - precision: 0.5678 - recall: 0.0494 - auc: 0.7233 - f1_score: 0.0901 - val_loss: 1.6254 - val_accuracy: 0.8596 - val_precision: 0.8301 - val_recall: 0.0220 - val_auc: 0.7553 - val_f1_score: 0.0420 - lr: 0.0010
Epoch 28/60
361/361 [==============================] - 325s 899ms/step - loss: 1.6867 - accuracy: 0.8592 - precision: 0.5863 - recall: 0.0480 - auc: 0.7245 - f1_score: 0.0879 - val_loss: 1.6093 - val_accuracy: 0.8627 - val_precision: 0.7092 - val_recall: 0.0656 - val_auc: 0.7608 - val_f1_score: 0.1176 - lr: 0.0010
Epoch 29/60
361/361 [==============================] - 321s 888ms/step - loss: 1.6847 - accuracy: 0.8590 - precision: 0.5723 - recall: 0.0515 - auc: 0.7247 - f1_score: 0.0936 - val_loss: 1.6246 - val_accuracy: 0.8619 - val_precision: 0.5929 - val_recall: 0.1069 - val_auc: 0.7537 - val_f1_score: 0.1879 - lr: 0.0010
Epoch 30/60
361/361 [==============================]

In [24]:
score = model2.predict(test_dataset)

111/111 [==============================] - 64s 570ms/step


111/111 [==============================] - 46s 412ms/step - loss: 1.6113 - accuracy: 0.8615 - precision: 0.7338 - recall: 0.0480 - auc: 0.7639 - f1_score: 0.0889
